In [ ]:
! pip install -q kaggle
from google.colab import files

files.upload()
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! mkdir Dataset
     

Saving kaggle.json to kaggle.json


In [ ]:
!kaggle datasets download -d utkarshsaxenadn/fast-food-classification-dataset


 99% 815M/821M [00:11<00:00, 73.2MB/s]
100% 821M/821M [00:11<00:00, 72.9MB/s]


In [ ]:
!kaggle datasets download -d kritikseth/fruit-and-vegetable-image-recognition
!kaggle datasets download -d trolukovich/food11-image-dataset

100% 1.98G/1.98G [00:24<00:00, 67.5MB/s]
100% 1.98G/1.98G [00:24<00:00, 87.1MB/s]
 99% 1.07G/1.08G [00:17<00:00, 137MB/s]
100% 1.08G/1.08G [00:17<00:00, 65.6MB/s]


In [ ]:
!unzip fast-food-classification-dataset  -d  fast-food-classification-dataset  
!unzip  fruit-and-vegetable-image-recognition -d  fruit-and-vegetable-image-recognition  
!unzip food11-image-dataset  -d  food11-image-dataset  


Streaming output truncated to the last 5000 lines.
  inflating: food11-image-dataset/training/Soup/223.jpg  
  inflating: food11-image-dataset/training/Soup/224.jpg  
  inflating: food11-image-dataset/training/Soup/225.jpg  
  inflating: food11-image-dataset/training/Soup/226.jpg  
  inflating: food11-image-dataset/training/Soup/227.jpg  
  inflating: food11-image-dataset/training/Soup/228.jpg  
  inflating: food11-image-dataset/training/Soup/229.jpg  
  inflating: food11-image-dataset/training/Soup/23.jpg  
  inflating: food11-image-dataset/training/Soup/230.jpg  
  inflating: food11-image-dataset/training/Soup/231.jpg  
  inflating: food11-image-dataset/training/Soup/232.jpg  
  inflating: food11-image-dataset/training/Soup/233.jpg  
  inflating: food11-image-dataset/training/Soup/234.jpg  
  inflating: food11-image-dataset/training/Soup/235.jpg  
  inflating: food11-image-dataset/training/Soup/236.jpg  
  inflating: food11-image-dataset/training/Soup/237.jpg  
  inflating: food11-im

In [ ]:
from distutils.dir_util import copy_tree

toDirectory='/content/food/train'
fromDirectory='/content/fast-food-classification-dataset/Fast Food Classification V2/Train'
copy_tree(fromDirectory,toDirectory)
#------------------------------------------------------------------------------
toDirectory='/content/food/val'
fromDirectory='/content/fast-food-classification-dataset/Fast Food Classification V2/Valid'


copy_tree(fromDirectory,toDirectory)
from distutils.dir_util import copy_tree

toDirectory='/content/food/train'
fromDirectory='/content/food11-image-dataset/training'
copy_tree(fromDirectory,toDirectory)
#------------------------------------------------------------------------------
toDirectory='/content/food/val'
fromDirectory='/content/food11-image-dataset/validation'


copy_tree(fromDirectory,toDirectory)
from distutils.dir_util import copy_tree

toDirectory='/content/food/train'
fromDirectory='/content/fruit-and-vegetable-image-recognition/train'
copy_tree(fromDirectory,toDirectory)
#------------------------------------------------------------------------------
toDirectory='/content/food/val'
fromDirectory='/content/fruit-and-vegetable-image-recognition/validation'


copy_tree(fromDirectory,toDirectory)

['/content/food/val/ginger/Image_10.jpg',
 '/content/food/val/ginger/Image_1.jpg',
 '/content/food/val/ginger/Image_8.jpg',
 '/content/food/val/ginger/Image_2.jpg',
 '/content/food/val/ginger/Image_3.jpg',
 '/content/food/val/ginger/Image_7.jpg',
 '/content/food/val/ginger/Image_4.jpg',
 '/content/food/val/ginger/Image_9.jpg',
 '/content/food/val/ginger/Image_6.jpg',
 '/content/food/val/ginger/Image_5.jpg',
 '/content/food/val/mango/Image_10.jpg',
 '/content/food/val/mango/Image_1.jpg',
 '/content/food/val/mango/Image_8.jpg',
 '/content/food/val/mango/Image_2.jpg',
 '/content/food/val/mango/Image_3.jpg',
 '/content/food/val/mango/Image_7.jpg',
 '/content/food/val/mango/Image_4.jpg',
 '/content/food/val/mango/Image_9.jpg',
 '/content/food/val/mango/Image_6.jpg',
 '/content/food/val/mango/Image_5.jpg',
 '/content/food/val/capsicum/Image_10.jpg',
 '/content/food/val/capsicum/Image_1.jpg',
 '/content/food/val/capsicum/Image_8.jpg',
 '/content/food/val/capsicum/Image_2.jpg',
 '/content/food

In [ ]:
import shutil

# delete directory recursively
shutil.rmtree('/content/food/train/Vegetable-Fruit')

shutil.rmtree('/content/food/val/Vegetable-Fruit')


In [ ]:
# Load the pre-trained MobileNetV3_large model and modify the last layer
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models


# Set device to GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the transforms to be applied to the images
model = models.mobilenet_v3_large(pretrained=True)
num_ftrs = model.classifier[-1].in_features
model.classifier[-1] = nn.Linear(num_ftrs, 56)
model.load_state_dict(torch.load("/content/drive/MyDrive/food.pt"))
# Set the model to device
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:


train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(size=224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
val_transforms = transforms.Compose([
    transforms.Resize(size=256),
    transforms.CenterCrop(size=224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the dataset
train_dataset = datasets.ImageFolder(root='/content/food/train', transform=train_transforms)
val_dataset = datasets.ImageFolder(root='/content/food/val', transform=val_transforms)

# Define the dataloaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)



# Train the model
num_epochs = 5
for epoch in range(num_epochs):
    # Set the model to train mode
    model.train()
    train_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # Compute the running training loss
        train_loss += loss.item() * inputs.size(0)
        
    # Compute the average training loss for the epoch
    train_loss = train_loss / len(train_loader.dataset)

    
    # Set the model to eval mode and compute the validation accuracy
    model.eval()
    num_correct = 0
    num_samples = 0
    
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            # Compute the number of correct predictions and samples
            num_correct += torch.sum(preds == labels)
            num_samples += labels.size(0)
    
    # Compute the validation accuracy for the epoch
    val_acc = num_correct.double() / num_samples
    # Set the model to eval mode and compute the train accuracy


    # Print the epoch number, training loss, training accuracy, and validation accuracy
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Acc:{val_acc:.4f}')








/usr/local/lib/python3.10/dist-packages/PIL/Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/5, Train Loss: 0.3565, Val Acc:0.9015
Epoch 2/5, Train Loss: 0.3685, Val Acc:0.8991
Epoch 3/5, Train Loss: 0.3605, Val Acc:0.9010
Epoch 4/5, Train Loss: 0.3551, Val Acc:0.9037
Epoch 5/5, Train Loss: 0.3501, Val Acc:0.9054


In [ ]:
num_epochs = 5
for epoch in range(num_epochs):
    # Set the model to train mode
    model.train()
    train_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # Compute the running training loss
        train_loss += loss.item() * inputs.size(0)
        
    # Compute the average training loss for the epoch
    train_loss = train_loss / len(train_loader.dataset)

    
    # Set the model to eval mode and compute the validation accuracy
    model.eval()
    num_correct = 0
    num_samples = 0
    
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            # Compute the number of correct predictions and samples
            num_correct += torch.sum(preds == labels)
            num_samples += labels.size(0)
    
    # Compute the validation accuracy for the epoch
    val_acc = num_correct.double() / num_samples
    # Set the model to eval mode and compute the train accuracy


    # Print the epoch number, training loss, training accuracy, and validation accuracy
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Acc:{val_acc:.4f}')



Epoch 1/5, Train Loss: 0.4015, Val Acc:0.8979
Epoch 2/5, Train Loss: 0.3921, Val Acc:0.8990
Epoch 3/5, Train Loss: 0.3880, Val Acc:0.9024
Epoch 4/5, Train Loss: 0.3803, Val Acc:0.9006
Epoch 5/5, Train Loss: 0.3752, Val Acc:0.8990


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/food.pt')


In [ ]:

toDirectory='/content/drive/MyDrive'
fromDirectory='/content/food'
copy_tree(fromDirectory,toDirectory)

['/content/drive/MyDrive/val/Hot Dog/Hot Dog-Valid (264).jpeg',
 '/content/drive/MyDrive/val/Hot Dog/Hot Dog-Valid (216).jpeg',
 '/content/drive/MyDrive/val/Hot Dog/Hot Dog-Valid (228).jpeg',
 '/content/drive/MyDrive/val/Hot Dog/Hot Dog-Valid (135).jpeg',
 '/content/drive/MyDrive/val/Hot Dog/Hot Dog-Valid (42).jpeg',
 '/content/drive/MyDrive/val/Hot Dog/Hot Dog-Valid (148).jpeg',
 '/content/drive/MyDrive/val/Hot Dog/Hot Dog-Valid (197).jpeg',
 '/content/drive/MyDrive/val/Hot Dog/Hot Dog-Valid (106).jpeg',
 '/content/drive/MyDrive/val/Hot Dog/Hot Dog-Valid (259).jpeg',
 '/content/drive/MyDrive/val/Hot Dog/Hot Dog-Valid (209).jpeg',
 '/content/drive/MyDrive/val/Hot Dog/Hot Dog-Valid (17).jpeg',
 '/content/drive/MyDrive/val/Hot Dog/Hot Dog-Valid (157).jpeg',
 '/content/drive/MyDrive/val/Hot Dog/Hot Dog-Valid (78).jpeg',
 '/content/drive/MyDrive/val/Hot Dog/Hot Dog-Valid (299).jpeg',
 '/content/drive/MyDrive/val/Hot Dog/Hot Dog-Valid (70).jpeg',
 '/content/drive/MyDrive/val/Hot Dog/Hot Dog

In [ ]:



import torchvision.transforms as transforms
from PIL import Image
image = Image.open('download.jpeg')

# Load the image and apply the same transforms as used during training and validation
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
image = transform(image)

# Add a batch dimension to the image tensor
image = image.unsqueeze(0)

# Pass the image through the trained model
model.eval()
with torch.no_grad():
    output = model(image.to(device))
    
# Get the predicted class index
_, predicted = torch.max(output.data, 1)
class_index = predicted.item()

# Print the predicted class label
print('Predicted class:', train_dataset.classes[class_index])

